In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import time
import os
import pickle

from tbpp_model import TBPP512, TBPP512_dense
from tbpp_utils import PriorUtil
from ssd_data import InputGenerator
from tbpp_training import TBPPFocalLoss
from utils.model import load_weights
from utils.training import Logger

Using TensorFlow backend.


### Data

In [2]:
from data_icdar2015fst import GTUtility
# with open('gt_util_synthtext.pkl', 'rb') as f:
#     gt_util = pickle.load(f)
gt_util_train = GTUtility('../data/icdar')
# gt_util_train, gt_util_val = gt_util.split(split=0.95)

### Model

In [ ]:
# TextBoxes++
model = TBPP512(softmax=False)
weights_path = 'models/ssd512_voc_weights_fixed.hdf5'
freeze = ['conv1_1', 'conv1_2',
          'conv2_1', 'conv2_2',
          'conv3_1', 'conv3_2', 'conv3_3',
          #'conv4_1', 'conv4_2', 'conv4_3',
          #'conv5_1', 'conv5_2', 'conv5_3',
         ]
batch_size = 24
experiment = 'tbpp512fl_synthtext'

In [3]:
# TextBoxes++ + DenseNet
model = TBPP512_dense(softmax=False)
weights_path = 'models/tbpp/weights.018.h5'
freeze = []
batch_size = 6
experiment = 'dsodtbpp512fl_synthtext'

In [4]:
prior_util = PriorUtil(model)

if weights_path is not None:
    load_weights(model, weights_path)

ssd_utils.py:210: RuntimeWarning: divide by zero encountered in double_scalars
  box_wh.append([min_size * np.sqrt(ar), min_size / np.sqrt(ar)])


layer missing max_pooling2d_9
    file  []


### Training

In [5]:
epochs = 100
initial_epoch = 0
batch_size = 3

gen_train = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size)
# gen_val = InputGenerator(gt_util_val, prior_util, batch_size, model.image_size)

for layer in model.layers:
    layer.trainable = not layer.name in freeze

checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment
if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())

#optim = keras.optimizers.SGD(lr=1e-3, momentum=0.9, decay=0, nesterov=True)
optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0)

# weight decay
regularizer = keras.regularizers.l2(5e-4) # None if disabled
regularizer = None
for l in model.layers:
    if l.__class__.__name__.startswith('Conv'):
        l.kernel_regularizer = regularizer

loss = TBPPFocalLoss()

model.compile(optimizer=optim, loss=loss.compute, metrics=loss.metrics)

history = model.fit_generator(
        gen_train.generate(),
        steps_per_epoch=int(gen_train.num_batches/4), 
        epochs=epochs, 
#         verbose=1, 
#         callbacks=[
#             keras.callbacks.ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', verbose=1, save_weights_only=True),
# #             Logger(checkdir),
#             #LearningRateDecay()
#         ], 
# #         validation_data=gen_val.generate(), 
# #         validation_steps=gen_val.num_batches, 
#         class_weight=None,
#         max_queue_size=1, 
#         workers=1, 
#         #use_multiprocessing=False, 
#         initial_epoch=initial_epoch, 
#         #pickle_safe=False, # will use threading instead of multiprocessing, which is lighter on memory use but slower
        )

Epoch 1/100


ssd_utils.py:37: RuntimeWarning: invalid value encountered in multiply
  area_gt = (priors[:, 2] - priors[:, 0]) * (priors[:, 3] - priors[:, 1])
tbpp_utils.py:43: RuntimeWarning: invalid value encountered in greater
  prior_mask = max_val > overlap_threshold


19/19 [==============================] - 26s 1s/step - loss: 3.3397 - conf_loss: 2.6509e-04 - loc_loss: 3.0746 - precision: 0.6876 - recall: 0.6590 - accuracy: 0.7805 - fmeasure: 0.6624 - num_pos: 111.0000 - num_neg: 229251.0000
Epoch 2/100
19/19 [==============================] - 6s 295ms/step - loss: 2.8189 - conf_loss: 2.1693e-04 - loc_loss: 2.6020 - precision: 0.6719 - recall: 0.6963 - accuracy: 0.8093 - fmeasure: 0.6624 - num_pos: 99.6842 - num_neg: 229262.3158
Epoch 3/100
19/19 [==============================] - 6s 295ms/step - loss: 2.9193 - conf_loss: 3.0755e-04 - loc_loss: 2.6117 - precision: 0.7373 - recall: 0.6216 - accuracy: 0.8035 - fmeasure: 0.6610 - num_pos: 110.0000 - num_neg: 229252.0000
Epoch 4/100
19/19 [==============================] - 6s 298ms/step - loss: 2.5308 - conf_loss: 3.0933e-04 - loc_loss: 2.2215 - precision: 0.7507 - recall: 0.7679 - accuracy: 0.7944 - fmeasure: 0.7457 - num_pos: 126.2632 - num_neg: 229235.7368
Epoch 5/100
19/19 [========================

19/19 [==============================] - 6s 299ms/step - loss: 0.5606 - conf_loss: 8.2768e-05 - loc_loss: 0.4779 - precision: 0.9156 - recall: 0.7852 - accuracy: 0.8939 - fmeasure: 0.8422 - num_pos: 113.3684 - num_neg: 229248.6316
Epoch 29/100
19/19 [==============================] - 6s 297ms/step - loss: 0.7249 - conf_loss: 1.2662e-04 - loc_loss: 0.5983 - precision: 0.9044 - recall: 0.7925 - accuracy: 0.9072 - fmeasure: 0.8305 - num_pos: 93.1579 - num_neg: 229268.8421
Epoch 30/100
19/19 [==============================] - 6s 296ms/step - loss: 0.4146 - conf_loss: 7.1468e-05 - loc_loss: 0.3431 - precision: 0.9177 - recall: 0.8188 - accuracy: 0.9193 - fmeasure: 0.8620 - num_pos: 100.6842 - num_neg: 229261.3158
Epoch 31/100
19/19 [==============================] - 6s 298ms/step - loss: 0.5874 - conf_loss: 7.4970e-05 - loc_loss: 0.5124 - precision: 0.8744 - recall: 0.7772 - accuracy: 0.9035 - fmeasure: 0.8185 - num_pos: 109.2105 - num_neg: 229252.7895
Epoch 32/100
19/19 [==================

19/19 [==============================] - 6s 296ms/step - loss: 0.2559 - conf_loss: 6.2750e-05 - loc_loss: 0.1932 - precision: 0.9568 - recall: 0.8405 - accuracy: 0.9244 - fmeasure: 0.8917 - num_pos: 122.4211 - num_neg: 229239.5789
Epoch 55/100
19/19 [==============================] - 6s 299ms/step - loss: 0.4290 - conf_loss: 9.1193e-05 - loc_loss: 0.3378 - precision: 0.9409 - recall: 0.7852 - accuracy: 0.9061 - fmeasure: 0.8449 - num_pos: 107.8421 - num_neg: 229254.1579
Epoch 56/100
19/19 [==============================] - 6s 297ms/step - loss: 0.2016 - conf_loss: 6.7154e-05 - loc_loss: 0.1345 - precision: 0.9337 - recall: 0.8277 - accuracy: 0.9165 - fmeasure: 0.8747 - num_pos: 119.7368 - num_neg: 229242.2632
Epoch 57/100
19/19 [==============================] - 6s 299ms/step - loss: 0.3924 - conf_loss: 9.6879e-05 - loc_loss: 0.2955 - precision: 0.9421 - recall: 0.7935 - accuracy: 0.9000 - fmeasure: 0.8545 - num_pos: 125.8421 - num_neg: 229236.1579
Epoch 58/100
19/19 [=================

19/19 [==============================] - 6s 298ms/step - loss: 0.3415 - conf_loss: 6.2261e-05 - loc_loss: 0.2792 - precision: 0.9457 - recall: 0.8305 - accuracy: 0.9279 - fmeasure: 0.8767 - num_pos: 99.8947 - num_neg: 229262.1053
Epoch 81/100
19/19 [==============================] - 6s 299ms/step - loss: 0.2237 - conf_loss: 4.9923e-05 - loc_loss: 0.1738 - precision: 0.9552 - recall: 0.8524 - accuracy: 0.9339 - fmeasure: 0.8993 - num_pos: 115.0526 - num_neg: 229246.9474
Epoch 82/100
19/19 [==============================] - 6s 297ms/step - loss: 0.2161 - conf_loss: 5.5061e-05 - loc_loss: 0.1611 - precision: 0.9379 - recall: 0.8504 - accuracy: 0.9281 - fmeasure: 0.8905 - num_pos: 115.4737 - num_neg: 229246.5263
Epoch 83/100
19/19 [==============================] - 6s 299ms/step - loss: 0.2285 - conf_loss: 6.8430e-05 - loc_loss: 0.1601 - precision: 0.9581 - recall: 0.8337 - accuracy: 0.9175 - fmeasure: 0.8877 - num_pos: 115.8421 - num_neg: 229246.1579
Epoch 84/100
19/19 [==================

from utils.model import calc_memory_usage, count_parameters
count_parameters(model)
calc_memory_usage(model)

In [ ]:
# frequency of class instance in local ground truth, used for weightning the focal loss
s = np.zeros(gt_util.num_classes)
for i in range(1000):#range(gt_util.num_samples):
    egt = prior_util.encode(gt_util.data[i])
    s += np.sum(egt[:,-gt_util.num_classes:], axis=0)
sn = np.asarray(np.sum(s))/s
print(np.array(sn, dtype=np.int32))
print(sn/np.sum(sn))